In [1]:
import escher
from escher import Builder
import cobra
from time import sleep
import os
import pickle

In [2]:
# Load the model into cobra
# Relative path to model file saved locally on Helen's computer, but no changes to the model since downloading it from BiGG
model = cobra.io.read_sbml_model('../../../../GEM-repos/mit1002-model/model.xml')

'' is not a valid SBML 'SId'.


In [16]:
model.reactions.EX_cpd00029_e0.reactants

[<Metabolite cpd00029_e0 at 0x7f9910630a90>]

In [19]:
# Get a list of reactions that involve acetate
ace_met_c = model.metabolites.cpd00029_c0
ace_met_e = model.metabolites.cpd00029_e0
ace_rxns = [r for r in model.reactions if ace_met_e in r.metabolites or ace_met_c in r.metabolites]

In [20]:
ace_rxns

[<Reaction rxn00225_c0 at 0x7f9931a74ee0>,
 <Reaction rxn00175_c0 at 0x7f99507c0c70>,
 <Reaction rxn05488_c0 at 0x7f99507ec070>,
 <Reaction rxn05733_c0 at 0x7f9950623940>,
 <Reaction rxn00649_c0 at 0x7f98f06138b0>,
 <Reaction rxn01484_c0 at 0x7f99413dbdf0>,
 <Reaction rxn07879_c0 at 0x7f99206c4970>,
 <Reaction rxn03146_c0 at 0x7f9941436eb0>,
 <Reaction rxn03504_c0 at 0x7f99015ce970>,
 <Reaction rxn08063_c0 at 0x7f99206d1940>,
 <Reaction rxn13143_c0 at 0x7f9901662f10>,
 <Reaction rxn00265_c0 at 0x7f99519618e0>,
 <Reaction EX_cpd00029_e0 at 0x7f9941493490>]

In [41]:
# Function to get all reactions one metabolite away from a given reaction
def get_neighbours(reaction):
    neighbours = []
    for met in reaction.metabolites:
        # Skip highly connected metabolites
        if met.id in ['cpd00001_c0',  # H2O
                      'cpd00002_c0',  # ATP
                      'cpd00003_c0',  # NAD+
                      'cpd00004_c0',  # NADH
                      'cpd00008_c0',  # ADP
                      'cpd00012_c0',  # PPi
                      'cpd00067_c0',  # H+
                      ]:
            continue
        for rxn in met.reactions:
            if rxn not in neighbours and rxn != reaction:
                neighbours.append(rxn)
    return neighbours

In [32]:
get_neighbours(model.reactions.EX_cpd00029_e0)

[<Reaction rxn08063_c0 at 0x7f99206d1940>,
 <Reaction rxn05488_c0 at 0x7f99507ec070>]

In [42]:
# Make a nested dictionary of the acetate reactions and their neighbours
ace_dict = {}
for rxn in ace_rxns:
    ace_dict[rxn.id] = get_neighbours(rxn)

In [43]:
# Print out the lengths of the nested dictionaries for each reaction
for k,v in ace_dict.items():
    print(k, len(v))

rxn00225_c0 16
rxn00175_c0 65
rxn05488_c0 61
rxn05733_c0 29
rxn00649_c0 23
rxn01484_c0 15
rxn07879_c0 11
rxn03146_c0 13
rxn03504_c0 11
rxn08063_c0 12
rxn13143_c0 21
rxn00265_c0 18
EX_cpd00029_e0 2


In [62]:
model.reactions.rxn00265_c0

Reaction identifier,rxn00265_c0
Name,citrate oxaloacetate-lyase (forming acetate from the pro-S carboxymethyl group of citrate)_c0
Memory address,0x7f99519618e0
Stoichiometry,cpd00137_c0 <-- cpd00029_c0 + cpd00032_c0 Citrate_c0 <-- Acetate_c0 + Oxaloacetate_c0
GPR,
Lower bound,-1000.0
Upper bound,0.0


In [3]:
# Load the solutions
# Load the results from the COBRA simulations
with open('results.pkl', 'rb') as f:
    cobra_results = pickle.load(f)

In [8]:
cobra_results['glc_medium_inf_o2_fba'].objective_value

1.1043992751974898

In [10]:
print('Infinite O2 (LB = -1000)\n------------------------\n')
print(f'Glucose (FBA): {cobra_results["glc_medium_inf_o2_fba"].objective_value}')
print(f'Acetate (FBA): {cobra_results["ace_medium_inf_o2_fba"].objective_value}')
print(f'Glucose Heavy Mixture (FBA): {cobra_results["glc_heavy_mix_medium_inf_o2_fba"].objective_value}')
print(f'Acetate Heavy Mixture (FBA): {cobra_results["ace_heavy_mix_medium_inf_o2_fba"].objective_value}')
print(f'Glucose (pFBA): {cobra_results["glc_medium_inf_o2_pfba"].fluxes.bio1_biomass}')
print(f'Acetate (pFBA): {cobra_results["ace_medium_inf_o2_pfba"].fluxes.bio1_biomass}')
print(f'Mixture (pFBA): {cobra_results["glc_heavy_mix_medium_inf_o2_pfba"].fluxes.bio1_biomass}')
print(f'Acetate Heavy Mixture (FBA): {cobra_results["ace_heavy_mix_medium_inf_o2_pfba"].fluxes.bio1_biomass}')
print('\nLimited O2 (LB = -20)\n---------------------\n')
print(f'Glucose (FBA): {cobra_results["glc_medium_real_o2_fba"].objective_value}')
print(f'Acetate (FBA): {cobra_results["ace_medium_real_o2_fba"].objective_value}')
print(f'Glucose Heavy Mixture (FBA): {cobra_results["glc_heavy_mix_medium_real_o2_fba"].objective_value}')
print(f'Acetate Heavy Mixture (FBA): {cobra_results["ace_heavy_mix_medium_real_o2_fba"].objective_value}')
print(f'Glucose (pFBA): {cobra_results["glc_medium_real_o2_pfba"].fluxes.bio1_biomass}')
print(f'Acetate (pFBA): {cobra_results["ace_medium_real_o2_pfba"].fluxes.bio1_biomass}')
print(f'Mixture (pFBA): {cobra_results["glc_heavy_mix_medium_real_o2_pfba"].fluxes.bio1_biomass}')
print(f'Acetate Heavy Mixture (FBA): {cobra_results["ace_heavy_mix_medium_real_o2_pfba"].fluxes.bio1_biomass}')

Infinite O2 (LB = -1000)
------------------------

Glucose (FBA): 1.1043992751974898
Acetate (FBA): 0.5244769419861662
Glucose Heavy Mixture (FBA): 0.9120647264925414
Acetate Heavy Mixture (FBA): 0.7196187348738703
Glucose (pFBA): 1.1043992751974805
Acetate (pFBA): 0.5244769419861675
Mixture (pFBA): 0.9120647264925411
Acetate Heavy Mixture (FBA): 0.7196187348738702

Limited O2 (LB = -20)
---------------------

Glucose (FBA): 1.1043992751974716
Acetate (FBA): 0.4883441468306618
Glucose Heavy Mixture (FBA): 0.9120647264925414
Acetate Heavy Mixture (FBA): 0.7196187348738694
Glucose (pFBA): 1.1043992751974663
Acetate (pFBA): 0.4883441468306319
Mixture (pFBA): 0.9120647264925411
Acetate Heavy Mixture (FBA): 0.7196187348738702


In [11]:
# Make a map with the solution for glucose only
glc_only_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = cobra_results["glc_medium_inf_o2_fba"].fluxes
)

In [12]:
# View the map
glc_only_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -3.5239759075541848, 'rxn02342_c0': 0.0, 'rxn06493_c…

In [13]:
model

Name,
Memory address,7f99406bc340
Number of metabolites,896
Number of reactions,881
Number of genes,331
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,"c0, e0"


In [50]:
# Make a map with the solution for glucose only
glc_only_full_map = Builder(
    map_json = '../../../../GEM-repos/mit1002-model/escher/e_coli.Core metabolism.modelseed.json',
    reaction_data = glc_only_fba.fluxes
)

In [51]:
glc_only_full_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -3.5239759075541848, 'rxn02342_c0': 0.0, 'rxn06493_c…

In [38]:
# Make a map with the solution for acetate only
ace_only_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = ace_only_fba.fluxes
)

In [39]:
ace_only_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -0.7524648226980373, 'rxn02342_c0': 0.0, 'rxn06493_c…

In [53]:
# Make a map with the solution for acetate only
ace_only_full_map = Builder(
    map_json = '../../../../GEM-repos/mit1002-model/escher/e_coli.Core metabolism.modelseed.json',
    reaction_data = ace_only_fba.fluxes
)

In [54]:
ace_only_full_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -0.7524648226980373, 'rxn02342_c0': 0.0, 'rxn06493_c…

In [40]:
glc_only_fba

,fluxes,reduced_costs
rxn00247_c0,0.000000,-6.368166e-03
rxn05625_c0,-3.523976,-2.649918e-17
rxn02342_c0,0.000000,-8.756229e-02
rxn06493_c0,0.025807,1.169186e-16
rxn02376_c0,0.000000,-1.133212e-18
...,...,...
EX_cpd00033_e0,0.000000,-2.220898e-01
EX_cpd00060_e0,0.000000,-3.128362e-01
DM_cpd15302_c0,0.000000,0.000000e+00
DM_cpd02701_c0,0.003099,0.000000e+00


In [41]:
# Make a map with the solution for glucose only using pFBA
glc_only_pfba_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = glc_only_pfba.fluxes
)

In [42]:
glc_only_pfba_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -3.523975907554188, 'rxn02342_c0': 0.0, 'rxn06493_c0…

In [43]:
# Make a map with the solution for acetate only using pFBA
ace_only_pfba_map = Builder(
    map_json = 'ace_transport_map.json',
    reaction_data = ace_only_pfba.fluxes
)

In [44]:
ace_only_pfba_map

Builder(reaction_data={'rxn00247_c0': 0.0, 'rxn05625_c0': -0.7524648226980581, 'rxn02342_c0': 0.0, 'rxn06493_c…